In [1]:
!pip install biopython
!apt-get update
!apt-get install -y ncbi-blast+

#  Installs BLAST+ tools and BioPython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.2 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,942 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRele

In [2]:
!blastdbcmd -version


blastdbcmd: 2.12.0+
 Package: blast 2.12.0, build Mar  8 2022 16:19:08


In [3]:

import os
import subprocess
import sqlite3
import pandas as pd
from Bio import SeqIO
import re
from google.colab import files

Upload Your Downloaded Files

In [6]:

print("Upload your BLAST database files:")
print("You need to upload:")
print("- All 18S_fungal_sequences.* files (ndb, nhr, nin, nsq, etc.)")
print("- taxdb.btd")
print("- taxdb.bti")
print("- taxonomy4blast.sqlite3")

# Upload files
uploaded = files.upload()

print(f"Uploaded files: {list(uploaded.keys())}")

# List all files in current directory
print("\nAll files in directory:")
for file in os.listdir("."):
    size = os.path.getsize(file) / 1024  # KB
    print(f"  {file} ({size:.1f} KB)")


Upload your BLAST database files:
You need to upload:
- All 18S_fungal_sequences.* files (ndb, nhr, nin, nsq, etc.)
- taxdb.btd
- taxdb.bti
- taxonomy4blast.sqlite3


Saving 18S_fungal_sequences.ndb to 18S_fungal_sequences (1).ndb
Saving 18S_fungal_sequences.nhr to 18S_fungal_sequences.nhr
Saving 18S_fungal_sequences.nin to 18S_fungal_sequences.nin
Saving 18S_fungal_sequences.nnd to 18S_fungal_sequences.nnd
Saving 18S_fungal_sequences.nni to 18S_fungal_sequences.nni
Saving 18S_fungal_sequences.nog to 18S_fungal_sequences.nog
Saving 18S_fungal_sequences.nos to 18S_fungal_sequences.nos
Saving 18S_fungal_sequences.not to 18S_fungal_sequences.not
Saving 18S_fungal_sequences.nsq to 18S_fungal_sequences.nsq
Saving 18S_fungal_sequences.ntf to 18S_fungal_sequences.ntf
Saving 18S_fungal_sequences.nto to 18S_fungal_sequences.nto
Saving taxdb.btd to taxdb.btd
Saving taxdb.bti to taxdb.bti
Saving taxonomy4blast.sqlite3 to taxonomy4blast.sqlite3
Uploaded files: ['18S_fungal_sequences (1).ndb', '18S_fungal_sequences.nhr', '18S_fungal_sequences.nin', '18S_fungal_sequences.nnd', '18S_fungal_sequences.nni', '18S_fungal_sequences.nog', '18S_fungal_sequences.nos', '18

In [7]:
def convert_blast_to_fasta(db_name="18S_fungal_sequences", output_file="18S_sequences.fasta"):
    """Convert BLAST database to FASTA format"""

    print(f"Converting {db_name} to FASTA format...")

    # Check if required database files exist
    required_extensions = ['.ndb', '.nhr', '.nin', '.nsq']
    required_files = [f"{db_name}{ext}" for ext in required_extensions]

    print("Checking for required database files:")
    missing_files = []
    for file in required_files:
        if os.path.exists(file):
            print(f"  ✓ {file}")
        else:
            print(f"  ❌ {file} (missing)")
            missing_files.append(file)

    if missing_files:
        print(f"\n❌ Missing files: {missing_files}")
        print("Please upload all the 18S_fungal_sequences.* files")
        return False

    try:
        # Run blastdbcmd to convert to FASTA
        print(f"\nRunning blastdbcmd...")

        result = subprocess.run([
            "blastdbcmd",
            "-db", db_name,
            "-entry", "all",
            "-outfmt", "%f",  # FASTA format
            "-out", output_file
        ], capture_output=True, text=True, check=True)

        print(f"✓ Conversion successful!")
        print(f"STDOUT: {result.stdout}")

        # Check output
        if os.path.exists(output_file):
            file_size = os.path.getsize(output_file) / (1024*1024)  # MB
            print(f"✓ Created {output_file} ({file_size:.2f} MB)")

            # Quick count of sequences
            with open(output_file, 'r') as f:
                seq_count = sum(1 for line in f if line.startswith('>'))
            print(f"✓ Contains {seq_count} sequences")

            return True
        else:
            print("❌ Output file not created")
            return False

    except subprocess.CalledProcessError as e:
        print(f"❌ blastdbcmd failed!")
        print(f"Error: {e}")
        print(f"STDERR: {e.stderr}")
        return False
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return False

# Run conversion
conversion_success = convert_blast_to_fasta()

Converting 18S_fungal_sequences to FASTA format...
Checking for required database files:
  ✓ 18S_fungal_sequences.ndb
  ✓ 18S_fungal_sequences.nhr
  ✓ 18S_fungal_sequences.nin
  ✓ 18S_fungal_sequences.nsq

Running blastdbcmd...
✓ Conversion successful!
STDOUT: 
✓ Created 18S_sequences.fasta (5.12 MB)
✓ Contains 3692 sequences


Load and Inspect FASTA Sequence

In [8]:
def load_and_inspect_fasta(fasta_file="18S_sequences.fasta", sample_size=20):
    """Load FASTA file and show detailed inspection"""

    if not os.path.exists(fasta_file):
        print(f"❌ {fasta_file} not found!")
        return None, None, None

    print(f"Loading sequences from {fasta_file}...")

    sequences = []
    seq_ids = []
    descriptions = []

    # Load all sequences
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequences.append(str(record.seq))
        seq_ids.append(record.id)
        descriptions.append(record.description)

    print(f"✓ Loaded {len(sequences)} sequences")

    # Detailed inspection
    print(f"\n🔍 DETAILED INSPECTION (First {sample_size} sequences):")
    print("=" * 100)

    for i in range(min(sample_size, len(sequences))):
        print(f"\n[{i+1}] SEQUENCE ID: {seq_ids[i]}")
        print(f"DESCRIPTION: {descriptions[i]}")
        print(f"LENGTH: {len(sequences[i])} bp")
        print(f"SEQUENCE: {sequences[i][:80]}{'...' if len(sequences[i]) > 80 else ''}")

        # Try to extract taxonomy from description
        genus_species = re.search(r'\b([A-Z][a-z]+)\s+([a-z]+)\b', descriptions[i])
        if genus_species:
            print(f"🏷️  EXTRACTED TAXA: {genus_species.group(1)} {genus_species.group(2)}")
        else:
            # Look for any capitalized words
            taxa_words = re.findall(r'\b[A-Z][a-z]{3,}\b', descriptions[i])
            if taxa_words:
                print(f"🏷️  POTENTIAL TAXA: {taxa_words[:3]}")
            else:
                print("🏷️  NO CLEAR TAXA FOUND")

        print("-" * 80)

    return sequences, seq_ids, descriptions

# Load and inspect sequences
if conversion_success:
    sequences, seq_ids, descriptions = load_and_inspect_fasta()


Loading sequences from 18S_sequences.fasta...
✓ Loaded 3692 sequences

🔍 DETAILED INSPECTION (First 20 sequences):

[1] SEQUENCE ID: NG_065155.1
DESCRIPTION: NG_065155.1 Zygosaccharomyces rouxii MUCL 30254 18S rRNA gene, partial sequence; from TYPE material
LENGTH: 1801 bp
SEQUENCE: TATCTGGTTGATCCTGCCAGTAGTCATATGCTTGTCTCAAAGATTAAGCCATGCATGTCTAAGTATAAGCAATTTATACA...
🏷️  EXTRACTED TAXA: Zygosaccharomyces rouxii
--------------------------------------------------------------------------------

[2] SEQUENCE ID: NG_065153.1
DESCRIPTION: NG_065153.1 Stagonospora bicolor ATCC 42652 18S rRNA gene, partial sequence; from TYPE material
LENGTH: 1756 bp
SEQUENCE: ATATGCTTGTCTCAGATTAAGCCATGCATGTCTAAGTATAAGCAATTATACCGTGAAACTGCGAATGGCTCATTAAATCA...
🏷️  EXTRACTED TAXA: Stagonospora bicolor
--------------------------------------------------------------------------------

[3] SEQUENCE ID: NG_062683.1
DESCRIPTION: NG_062683.1 Taphrina populina CBS 337.55 18S rRNA gene, partial sequence; from TYPE material

headers clearly include binomial names for many entries, so description parsing will yield a sizable labeled subset for supervised PEFT and for validating unsupervised clusters



In [13]:
# CELL 5: Explore Taxonomy SQLite Database

def detailed_sqlite_exploration(sqlite_file="taxonomy4blast.sqlite3"):
    """Detailed exploration of taxonomy SQLite database"""

    if not os.path.exists(sqlite_file):
        print(f"❌ {sqlite_file} not found!")
        return

    print(f"🔍 EXPLORING {sqlite_file}")
    print("=" * 60)

    try:
        conn = sqlite3.connect(sqlite_file)
        cursor = conn.cursor()

        # Get database info
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = [table[0] for table in cursor.fetchall()]
        print(f"📋 TABLES FOUND: {tables}")

        # Explore each table in detail
        for table_name in tables:
            print(f"\n🔍 TABLE: {table_name}")
            print("-" * 40)

            # Get column structure
            cursor.execute(f"PRAGMA table_info({table_name})")
            columns = cursor.fetchall()
            print("COLUMNS:")
            for col in columns:
                print(f"  {col[1]} ({col[2]})")  # name (type)

            # Get row count
            cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
            row_count = cursor.fetchone()[0]
            print(f"ROWS: {row_count}")

            # Get sample data
            cursor.execute(f"SELECT * FROM {table_name} LIMIT 5")
            sample_rows = cursor.fetchall()

            if sample_rows:
                print("SAMPLE DATA:")
                column_names = [col[1] for col in columns]
                for i, row in enumerate(sample_rows):
                    print(f"  Row {i+1}:")
                    for col_name, value in zip(column_names, row):
                        print(f"    {col_name}: {value}")
                    print()

        conn.close()

    except Exception as e:
        print(f"❌ Error exploring SQLite: {e}")

# Explore taxonomy database
detailed_sqlite_exploration()

🔍 EXPLORING taxonomy4blast.sqlite3
📋 TABLES FOUND: ['TaxidInfo']

🔍 TABLE: TaxidInfo
----------------------------------------
COLUMNS:
  taxid (INTEGER)
  parent (INTEGER)
ROWS: 3037025
SAMPLE DATA:
  Row 1:
    taxid: 1
    parent: 1

  Row 2:
    taxid: 2
    parent: 131567

  Row 3:
    taxid: 6
    parent: 335928

  Row 4:
    taxid: 7
    parent: 6

  Row 5:
    taxid: 9
    parent: 32199



# CELL 6: Extract Labels Using Best Method


In [10]:
def extract_comprehensive_labels(seq_ids, descriptions, sqlite_file="taxonomy4blast.sqlite3"):
    """Extract labels using the most successful method"""

    print("🎯 COMPREHENSIVE LABEL EXTRACTION")
    print("=" * 50)

    # Method 1: FASTA Description Parsing
    print("\n📝 METHOD 1: Parsing FASTA Descriptions")
    desc_labels = {}
    genera_found = set()

    for seq_id, desc in zip(seq_ids, descriptions):
        # Extract scientific names
        scientific_match = re.search(r'\b([A-Z][a-z]+)\s+([a-z]+)\b', desc)
        if scientific_match:
            genus = scientific_match.group(1)
            species = scientific_match.group(2)
            desc_labels[seq_id] = {
                'genus': genus,
                'species': species,
                'full_name': f"{genus} {species}",
                'source': 'description'
            }
            genera_found.add(genus)

    print(f"✓ Extracted from descriptions: {len(desc_labels)} sequences")
    print(f"✓ Unique genera found: {len(genera_found)}")

    # Method 2: SQLite Database (if available)
    sqlite_labels = {}
    if os.path.exists(sqlite_file):
        print(f"\n💾 METHOD 2: SQLite Database Query")
        try:
            conn = sqlite3.connect(sqlite_file)
            cursor = conn.cursor()

            # Try to find the right table and column structure
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = [t[0] for t in cursor.fetchall()]

            for table in tables:
                print(f"  Trying table: {table}")
                try:
                    cursor.execute(f"SELECT * FROM {table} LIMIT 1")
                    sample = cursor.fetchone()
                    if sample:
                        print(f"    Sample row: {sample}")
                except:
                    continue

            conn.close()

        except Exception as e:
            print(f"  SQLite query failed: {e}")

    # Combine results
    print(f"\n📊 FINAL RESULTS:")
    print(f"Description-based labels: {len(desc_labels)}")
    print(f"SQLite-based labels: {len(sqlite_labels)}")

    # Use description labels (more reliable for this dataset)
    final_labels = desc_labels

    # Create summary DataFrame
    if final_labels:
        label_df = pd.DataFrame([
            {
                'seq_id': seq_id,
                'genus': info['genus'],
                'species': info['species'],
                'full_name': info['full_name']
            }
            for seq_id, info in final_labels.items()
        ])

        print(f"\n📋 LABEL SUMMARY:")
        print(f"Total labeled sequences: {len(label_df)}")
        print(f"Unique genera: {label_df['genus'].nunique()}")
        print(f"Most common genera:")
        print(label_df['genus'].value_counts().head(10))

        return final_labels, label_df
    else:
        print("❌ No labels extracted successfully")
        return {}, pd.DataFrame()

# Extract comprehensive labels
if conversion_success and sequences:
    final_labels, label_df = extract_comprehensive_labels(seq_ids, descriptions)

🎯 COMPREHENSIVE LABEL EXTRACTION

📝 METHOD 1: Parsing FASTA Descriptions
✓ Extracted from descriptions: 3628 sequences
✓ Unique genera found: 1433

💾 METHOD 2: SQLite Database Query
  Trying table: TaxidInfo
    Sample row: (1, 1)

📊 FINAL RESULTS:
Description-based labels: 3628
SQLite-based labels: 0

📋 LABEL SUMMARY:
Total labeled sequences: 3628
Unique genera: 1433
Most common genera:
genus
Mucor              49
Candida            35
Alternaria         31
Exophiala          29
Ogataea            28
Scolecobasidium    27
Lophiostoma        24
Pichia             22
Ophiocordyceps     22
Aspergillus        22
Name: count, dtype: int64


Strong result—3,628 of 3,692 sequences now have parsed genus/species labels, giving a large supervised set plus a small unlabeled remainder for discovery and semi‑supervised loops.

Build supervised training targets at multiple ranks by collapsing the parsed binomials to genus, family, order, etc., after mapping names to taxids and ranks; this enables macro‑F1 at genus/species and robustness at higher ranks

Retain 64 unlabeled sequences for the unsupervised branch; they can seed novel‑taxon clusters with UMAP→HDBSCAN and later be pseudo‑labeled via cluster consensus BLAST or majority label propagation

In [11]:
print("\n" + "="*60)
print("🎉 BLAST CONVERSION & LABEL EXTRACTION COMPLETE!")
print("="*60)

if conversion_success:
    print(f"✅ FASTA Conversion: SUCCESS")
    print(f"   📁 Output file: 18S_sequences.fasta")
    print(f"   📊 Total sequences: {len(sequences)}")
    print(f"   📏 Average sequence length: {sum(len(s) for s in sequences)/len(sequences):.1f} bp")

    if final_labels:
        print(f"✅ Label Extraction: SUCCESS")
        print(f"   🏷️  Labeled sequences: {len(final_labels)}")
        print(f"   🧬 Unique genera: {len(set(info['genus'] for info in final_labels.values()))}")
        print(f"   📈 Label coverage: {len(final_labels)/len(sequences)*100:.1f}%")

        print(f"\n🔬 EXAMPLE EXTRACTED DATA:")
        for i, (seq_id, seq) in enumerate(zip(seq_ids[:3], sequences[:3])):
            print(f"\nSequence {i+1}:")
            print(f"  ID: {seq_id}")
            print(f"  Length: {len(seq)} bp")
            if seq_id in final_labels:
                print(f"  Genus: {final_labels[seq_id]['genus']}")
                print(f"  Species: {final_labels[seq_id]['species']}")
            else:
                print(f"  Label: Not found")
            print(f"  Sequence: {seq[:60]}...")
    else:
        print(f"❌ Label Extraction: FAILED")
        print("   Try manual inspection of file descriptions")
else:
    print(f"❌ FASTA Conversion: FAILED")
    print("   Check if all required .ndb, .nhr, .nin, .nsq files are uploaded")

print(f"\n🚀 READY FOR NEXT STEPS:")
if conversion_success and final_labels:
    print("   ✅ You can now proceed with UMAP/HDBSCAN clustering")
    print("   ✅ You can proceed with LoRA fine-tuning")
    print(f"   📊 You have {len(final_labels)} labeled sequences for supervised learning")
    print(f"   🔍 You have {len(sequences) - len(final_labels)} unlabeled sequences for novel discovery")
elif conversion_success:
    print("   ✅ You can proceed with unsupervised UMAP/HDBSCAN only")
    print("   ❌ No supervised learning possible without labels")
else:
    print("   ❌ Fix file upload and conversion issues first")



🎉 BLAST CONVERSION & LABEL EXTRACTION COMPLETE!
✅ FASTA Conversion: SUCCESS
   📁 Output file: 18S_sequences.fasta
   📊 Total sequences: 3692
   📏 Average sequence length: 1333.0 bp
✅ Label Extraction: SUCCESS
   🏷️  Labeled sequences: 3628
   🧬 Unique genera: 1433
   📈 Label coverage: 98.3%

🔬 EXAMPLE EXTRACTED DATA:

Sequence 1:
  ID: NG_065155.1
  Length: 1801 bp
  Genus: Zygosaccharomyces
  Species: rouxii
  Sequence: TATCTGGTTGATCCTGCCAGTAGTCATATGCTTGTCTCAAAGATTAAGCCATGCATGTCT...

Sequence 2:
  ID: NG_065153.1
  Length: 1756 bp
  Genus: Stagonospora
  Species: bicolor
  Sequence: ATATGCTTGTCTCAGATTAAGCCATGCATGTCTAAGTATAAGCAATTATACCGTGAAACT...

Sequence 3:
  ID: NG_062683.1
  Length: 1744 bp
  Genus: Taphrina
  Species: populina
  Sequence: CTTGTCTCAAAGATTAAGCCATGCATGTCTAAGTATAAGCAATTTATACAGTGAAACTGC...

🚀 READY FOR NEXT STEPS:
   ✅ You can now proceed with UMAP/HDBSCAN clustering
   ✅ You can proceed with LoRA fine-tuning
   📊 You have 3628 labeled sequences for supervised learnin

this dataset is ready for both supervised PEFT fine-tuning and unsupervised discovery with UMAP→HDBSCAN.

In [12]:
# Save processed data
if conversion_success:
    # Save sequences as CSV for easy loading
    df = pd.DataFrame({
        'seq_id': seq_ids,
        'sequence': sequences,
        'description': descriptions,
        'sequence_length': [len(s) for s in sequences]
    })

    # Add labels if available
    if final_labels:
        df['genus'] = df['seq_id'].map(lambda x: final_labels.get(x, {}).get('genus', 'Unknown'))
        df['species'] = df['seq_id'].map(lambda x: final_labels.get(x, {}).get('species', 'Unknown'))
        df['has_label'] = df['seq_id'].map(lambda x: x in final_labels)

    # Save to CSV
    df.to_csv('edna_sequences_processed.csv', index=False)
    print(f"✅ Saved processed data to 'edna_sequences_processed.csv'")
    print(f"📊 Dataset shape: {df.shape}")

    # Show data preview
    print(f"\n📋 DATA PREVIEW:")
    print(df.head())

    if final_labels:
        print(f"\n🏷️  LABEL DISTRIBUTION:")
        label_counts = df[df['has_label']]['genus'].value_counts().head(10)
        print(label_counts)

print(f"\n🎯 NEXT STEPS:")
print("1. Run UMAP + HDBSCAN on the sequences for novel taxa discovery")
print("2. If you have labels, set up LoRA fine-tuning")
print("3. Combine results for comprehensive biodiversity assessment")

✅ Saved processed data to 'edna_sequences_processed.csv'
📊 Dataset shape: (3692, 7)

📋 DATA PREVIEW:
        seq_id                                           sequence  \
0  NG_065155.1  TATCTGGTTGATCCTGCCAGTAGTCATATGCTTGTCTCAAAGATTA...   
1  NG_065153.1  ATATGCTTGTCTCAGATTAAGCCATGCATGTCTAAGTATAAGCAAT...   
2  NG_062683.1  CTTGTCTCAAAGATTAAGCCATGCATGTCTAAGTATAAGCAATTTA...   
3  NG_063110.1  GCGGTAATTCCAGCTCCAATAGCGTATATTAAAGTTGTTGCAGTTA...   
4  NG_063111.1  GCGGTAATTCCAGCTCCAATAGCGTATATTAAAGTTGTTGCAGTTA...   

                                         description  sequence_length  \
0  NG_065155.1 Zygosaccharomyces rouxii MUCL 3025...             1801   
1  NG_065153.1 Stagonospora bicolor ATCC 42652 18...             1756   
2  NG_062683.1 Taphrina populina CBS 337.55 18S r...             1744   
3  NG_063110.1 Epidermophyton floccosum CBS 230.7...              845   
4  NG_063111.1 Trichophyton rubrum CBS 392.58 18S...              844   

               genus    species  has_label  
